# 🚀 Phase 1: Google Colab 환경 구성

## 이 노트북의 목표
1. Google Colab에서 GPU 설정하기
2. 필수 라이브러리 설치
3. Google Drive 연동 (데이터/모델 저장용)
4. 환경 테스트

---
## ⚠️ 시작하기 전에

### Colab에서 GPU 설정하는 법
1. 상단 메뉴: **런타임(Runtime) → 런타임 유형 변경(Change runtime type)**
2. **하드웨어 가속기(Hardware accelerator)**: T4 GPU 선택
3. **저장(Save)** 클릭

### 무료 vs 유료
- **무료**: T4 GPU (16GB VRAM) - 하루 사용 제한 있음
- **Colab Pro**: 더 긴 세션, 더 좋은 GPU (A100 가능)
- **우리 프로젝트**: 무료 T4로 충분합니다! ✅

## 1️⃣ GPU 설정 확인

In [6]:
import torch
import sys

print("="*50)
print("🔍 환경 정보")
print("="*50)
print(f"Python 버전: {sys.version.split()[0]}")
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 버전: {torch.version.cuda}")
print()

print("="*50)
print("🎮 GPU 정보")
print("="*50)
if torch.cuda.is_available():
    print(f"✅ GPU 사용 가능!")
    print(f"GPU 이름: {torch.cuda.get_device_name(0)}")
    print(f"GPU 개수: {torch.cuda.device_count()}")
    
    # GPU 메모리 정보
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"전체 GPU 메모리: {gpu_memory:.2f} GB")
    
    # 사용 가능한 메모리
    free_memory = torch.cuda.mem_get_info()[0] / 1e9
    print(f"사용 가능 메모리: {free_memory:.2f} GB")
    print()
    print("💡 T4 GPU (16GB)면 7B 모델 QLoRA 학습 가능합니다!")
else:
    print("❌ GPU를 사용할 수 없습니다!")
    print("\n해결 방법:")
    print("1. 상단 메뉴: 런타임 → 런타임 유형 변경")
    print("2. 하드웨어 가속기: T4 GPU 선택")
    print("3. 저장 후 이 셀을 다시 실행하세요")

🔍 환경 정보
Python 버전: 3.13.2
PyTorch 버전: 2.9.0
CUDA 버전: None

🎮 GPU 정보
❌ GPU를 사용할 수 없습니다!

해결 방법:
1. 상단 메뉴: 런타임 → 런타임 유형 변경
2. 하드웨어 가속기: T4 GPU 선택
3. 저장 후 이 셀을 다시 실행하세요


## 2️⃣ Google Drive 연동

학습한 모델과 데이터를 저장하기 위해 Google Drive를 연결합니다.

In [7]:
from google.colab import drive

# Drive 마운트
drive.mount('/content/drive')

print("\n✅ Google Drive 연동 완료!")
print("📁 경로: /content/drive/MyDrive/")

ModuleNotFoundError: No module named 'google.colab'

## 3️⃣ 프로젝트 디렉토리 설정

Drive에 프로젝트 폴더를 만들어 학습 결과를 저장합니다.

In [ ]:
import os

# 프로젝트 경로 설정
PROJECT_NAME = "chatbot_project"
DRIVE_PROJECT_PATH = f"/content/drive/MyDrive/{PROJECT_NAME}"

# 필요한 디렉토리 생성
directories = [
    f"{DRIVE_PROJECT_PATH}/data/raw",
    f"{DRIVE_PROJECT_PATH}/data/processed",
    f"{DRIVE_PROJECT_PATH}/models/checkpoints",
    f"{DRIVE_PROJECT_PATH}/models/final",
    f"{DRIVE_PROJECT_PATH}/outputs/logs",
]

for directory in directories:
    os.makedirs(directory, exist_ok=True)
    print(f"✅ 생성: {directory}")

print(f"\n📁 프로젝트 루트: {DRIVE_PROJECT_PATH}")
print("\n💡 학습 중 세션이 끊겨도 Drive에 저장되므로 안전합니다!")

## 4️⃣ 필수 라이브러리 설치

### 설치 시간: 약 3-5분 ⏱️
- Colab은 매번 새로운 환경이므로 실행할 때마다 설치해야 합니다.
- 한번만 실행하면 됩니다!

In [ ]:
%%capture
# %%capture: 설치 로그를 숨깁니다 (속도는 동일)
# 로그를 보고 싶으면 이 줄을 삭제하세요

# 핵심 라이브러리
!pip install -q transformers>=4.35.0
!pip install -q datasets>=2.14.0
!pip install -q peft>=0.6.0
!pip install -q bitsandbytes>=0.41.0
!pip install -q accelerate>=0.24.0
!pip install -q sentencepiece>=0.1.99
!pip install -q protobuf>=3.20.0

print("✅ 모든 라이브러리 설치 완료!")

## 5️⃣ 설치 확인

In [ ]:
# 필수 라이브러리 import 테스트
print("📦 라이브러리 버전 확인\n")

try:
    import transformers
    import datasets
    import peft
    import bitsandbytes as bnb
    import accelerate
    
    print(f"✅ transformers: {transformers.__version__}")
    print(f"✅ datasets: {datasets.__version__}")
    print(f"✅ peft: {peft.__version__}")
    print(f"✅ bitsandbytes: {bnb.__version__}")
    print(f"✅ accelerate: {accelerate.__version__}")
    
    print("\n🎉 모든 라이브러리가 정상적으로 설치되었습니다!")
    
except ImportError as e:
    print(f"❌ 오류 발생: {e}")
    print("위의 설치 셀을 다시 실행해주세요.")

## 6️⃣ 간단한 모델 테스트

작은 한국어 모델로 로딩/생성이 정상적으로 작동하는지 확인합니다.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

print("🔄 테스트 모델 로딩 중... (약 30초 소요)\n")

# 작은 한국어 모델 (약 250MB)
model_name = "skt/kogpt2-base-v2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# GPU로 이동
if torch.cuda.is_available():
    model = model.to("cuda")
    print("✅ 모델을 GPU로 이동했습니다.")

print(f"✅ 모델 로딩 완료!")
print(f"📊 파라미터 수: {model.num_parameters():,}개\n")

# 간단한 생성 테스트
test_prompt = "인공지능의 미래는"
inputs = tokenizer(test_prompt, return_tensors="pt")

if torch.cuda.is_available():
    inputs = {k: v.to("cuda") for k, v in inputs.items()}

print("🔄 텍스트 생성 중...\n")
outputs = model.generate(
    **inputs,
    max_length=50,
    num_return_sequences=1,
    temperature=0.8,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

generated = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("="*50)
print(f"입력: {test_prompt}")
print(f"생성: {generated}")
print("="*50)
print("\n✅ 텍스트 생성 성공! 환경이 정상적으로 구성되었습니다.")

## 7️⃣ 메모리 정리 (선택사항)

테스트가 끝났으면 메모리를 정리합니다.

In [ ]:
# 모델 삭제
del model
del tokenizer

# GPU 메모리 정리
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    
print("✅ 메모리 정리 완료")

---

## ✅ Phase 1 완료 체크리스트

- [ ] GPU (T4) 설정 완료
- [ ] GPU 메모리 확인 (약 15GB 사용 가능)
- [ ] Google Drive 연동 완료
- [ ] 프로젝트 디렉토리 생성 완료
- [ ] 필수 라이브러리 설치 완료
- [ ] 모델 로딩/생성 테스트 성공

---

## 🎯 다음 단계: Phase 2

이제 환경 구성이 끝났습니다! 다음에는:

1. **데이터 준비**: 학습용 한국어 대화 데이터 만들기
2. **데이터 포맷**: Chat template 형식으로 변환
3. **데이터 업로드**: Google Drive에 저장

---

## 💡 Colab 사용 팁

### 세션 유지하기
- 무료 Colab은 약 12시간 후 세션이 끊깁니다
- 학습 중에는 탭을 닫지 마세요
- Google Drive에 자동 저장되므로 체크포인트는 안전합니다

### 메모리 부족 시
```python
# 메모리 정리
import gc
gc.collect()
torch.cuda.empty_cache()
```

### GPU 사용량 모니터링
```python
!nvidia-smi
```

### 중요 파일은 Drive에 저장
- `/content/`는 세션이 끊기면 삭제됩니다
- `/content/drive/MyDrive/`에 저장하세요